# Arquivos Indexados e Manipulação de Tabelas em Python

Dados raramente vivem apenas na memória — precisamos **salvar**, **recuperar** e **organizar** informações em arquivos. Nesta aula, vamos aprender a:

1. **Arquivos de texto** — ler e escrever dados em arquivos `.txt`
2. **Arquivos CSV** — ler e escrever tabelas com o módulo `csv`
3. **Acesso indexado** — buscar registros por chave (ID, código, etc.)
4. **Arquivos JSON** — armazenar dados estruturados com o módulo `json`
5. **Manipulação de tabelas** — filtrar, ordenar e agregar dados
6. **Projeto prático** — sistema de cadastro completo com arquivo

> **Pré-requisitos:** funções, listas, dicionários e laços de repetição.

---
## 1. Leitura e Escrita de Arquivos de Texto

Python usa a função `open()` para abrir arquivos. O parâmetro `mode` define o que faremos:

| Modo | Descrição |
|------|-----------|
| `'r'` | Leitura (**r**ead) — padrão |
| `'w'` | Escrita (**w**rite) — cria ou sobrescreve |
| `'a'` | Acréscimo (**a**ppend) — adiciona ao final |
| `'x'` | Criação exclusiva — erro se o arquivo já existir |

Sempre use `with open(...)` — ele fecha o arquivo automaticamente ao sair do bloco.

```python
with open('arquivo.txt', 'r', encoding='utf-8') as f:
    conteudo = f.read()
```

In [ ]:
# Escrevendo em um arquivo de texto
with open('notas.txt', 'w', encoding='utf-8') as f:
    f.write('Ana: 8.5\n')
    f.write('Bruno: 6.0\n')
    f.write('Carlos: 9.2\n')
    f.write('Diana: 7.0\n')

print('Arquivo criado com sucesso!')

In [ ]:
# Lendo o arquivo inteiro
with open('notas.txt', 'r', encoding='utf-8') as f:
    conteudo = f.read()

print(conteudo)

In [ ]:
# Lendo linha por linha com readlines()
with open('notas.txt', 'r', encoding='utf-8') as f:
    linhas = f.readlines()   # retorna lista de strings

print(f'Número de linhas: {len(linhas)}')
for linha in linhas:
    linha = linha.strip()    # remove \n e espaços
    print(f'  → {linha}')

In [ ]:
# Iterando diretamente sobre o arquivo (mais eficiente)
with open('notas.txt', 'r', encoding='utf-8') as f:
    for linha in f:
        nome, nota = linha.strip().split(': ')
        nota = float(nota)
        situacao = 'Aprovado' if nota >= 7 else 'Reprovado'
        print(f'{nome:<10} {nota:.1f}  →  {situacao}')

In [ ]:
# Modo append: adiciona sem apagar o que existe
with open('notas.txt', 'a', encoding='utf-8') as f:
    f.write('Eduardo: 8.0\n')
    f.write('Fernanda: 5.5\n')

# Verificando o resultado
with open('notas.txt', 'r', encoding='utf-8') as f:
    print(f.read())

---
## 2. Arquivos CSV — Tabelas em Arquivo

**CSV** (*Comma-Separated Values*) é o formato mais comum para tabelas em texto puro. Cada linha é um registro, cada coluna é separada por vírgula (ou ponto-e-vírgula):

```
id,nome,nota,cidade
1,Ana,8.5,Rio de Janeiro
2,Bruno,6.0,São Paulo
3,Carlos,9.2,Belo Horizonte
```

O módulo `csv` do Python facilita a leitura e escrita desse formato.

In [ ]:
import csv

# Criando um arquivo CSV com csv.writer
alunos = [
    ['id', 'nome',     'nota', 'cidade'],
    [1,    'Ana',       8.5,   'Rio de Janeiro'],
    [2,    'Bruno',     6.0,   'São Paulo'],
    [3,    'Carlos',    9.2,   'Belo Horizonte'],
    [4,    'Diana',     7.0,   'Rio de Janeiro'],
    [5,    'Eduardo',   8.0,   'Curitiba'],
    [6,    'Fernanda',  5.5,   'São Paulo'],
    [7,    'Gabriel',   9.8,   'Rio de Janeiro'],
    [8,    'Helena',    4.0,   'Curitiba'],
]

with open('alunos.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(alunos)   # escreve todas as linhas de uma vez

print('Arquivo alunos.csv criado!')

In [ ]:
# Lendo com csv.reader (retorna listas)
with open('alunos.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for linha in reader:
        print(linha)

### 2.1 DictReader e DictWriter — CSV com Dicionários

O `DictReader` usa o cabeçalho como chave, retornando **dicionários** em vez de listas. Isso torna o código muito mais legível — acessamos `linha['nome']` em vez de `linha[1]`.

In [ ]:
# DictReader: cada linha vira um dicionário
with open('alunos.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for aluno in reader:
        print(f"ID {aluno['id']:>2} | {aluno['nome']:<10} | Nota: {aluno['nota']} | {aluno['cidade']}")

In [ ]:
# Carregando CSV em uma lista de dicionários (tabela em memória)
def carregar_csv(caminho):
    """Lê um CSV e retorna uma lista de dicionários."""
    with open(caminho, 'r', encoding='utf-8') as f:
        return list(csv.DictReader(f))

alunos = carregar_csv('alunos.csv')

print(f'Total de registros: {len(alunos)}')
print(f'Primeiro registro:  {alunos[0]}')
print(f'Colunas:            {list(alunos[0].keys())}')

In [ ]:
# DictWriter: escrever dicionários no CSV
novos_alunos = [
    {'id': 9,  'nome': 'Igor',    'nota': 7.5, 'cidade': 'Brasília'},
    {'id': 10, 'nome': 'Juliana', 'nota': 8.8, 'cidade': 'São Paulo'},
]

with open('alunos.csv', 'a', newline='', encoding='utf-8') as f:
    campos = ['id', 'nome', 'nota', 'cidade']
    writer = csv.DictWriter(f, fieldnames=campos)
    writer.writerows(novos_alunos)

alunos = carregar_csv('alunos.csv')
print(f'Total após inclusão: {len(alunos)} registros')

---
## 3. Acesso Indexado — Busca por Chave

Um **arquivo indexado** permite acessar registros diretamente pela chave (ID, CPF, código), sem percorrer todos os registros. Em Python, simulamos isso carregando os dados em um **dicionário** onde a chave é o identificador único:

```
tabela_csv          →   dicionário indexado
─────────────────       ─────────────────────
id | nome | nota        {
 1 | Ana  | 8.5           '1': {'nome':'Ana',   'nota':'8.5'},
 2 | Bruno| 6.0           '2': {'nome':'Bruno', 'nota':'6.0'},
```

A busca passa de **O(n)** (percorrer a lista) para **O(1)** (acesso direto pelo dicionário).

In [ ]:
# Construindo índice por ID a partir do CSV
def construir_indice(caminho, chave):
    """Lê CSV e indexa os registros pelo campo indicado."""
    with open(caminho, 'r', encoding='utf-8') as f:
        return {linha[chave]: linha for linha in csv.DictReader(f)}

indice = construir_indice('alunos.csv', 'id')

# Busca direta por ID — acesso O(1)
aluno = indice.get('3')
if aluno:
    print(f"Encontrado: {aluno}")
else:
    print('Aluno não encontrado.')

In [ ]:
# Busca por nome (índice secundário)
indice_nome = construir_indice('alunos.csv', 'nome')

for nome_busca in ['Ana', 'Carlos', 'Maria']:
    resultado = indice_nome.get(nome_busca)
    if resultado:
        print(f"'{nome_busca}' → ID {resultado['id']}, Nota: {resultado['nota']}, Cidade: {resultado['cidade']}")
    else:
        print(f"'{nome_busca}' → não encontrado")

In [ ]:
# Função de busca genérica — percorre a lista buscando qualquer campo
def buscar(tabela, campo, valor):
    """Retorna todos os registros onde campo == valor."""
    return [r for r in tabela if r[campo].lower() == valor.lower()]

alunos = carregar_csv('alunos.csv')

resultado = buscar(alunos, 'cidade', 'São Paulo')
print(f"Alunos de São Paulo ({len(resultado)}):")
for a in resultado:
    print(f"  {a['nome']} — nota {a['nota']}")

---
## 4. Arquivos JSON — Dados Estruturados

**JSON** (*JavaScript Object Notation*) é um formato de texto para dados estruturados, muito usado em APIs e configurações. Suporta:

| JSON | Python |
|------|--------|
| `object {}` | `dict` |
| `array []` | `list` |
| `string ""` | `str` |
| `number` | `int` / `float` |
| `true/false` | `True/False` |
| `null` | `None` |

O módulo `json` converte entre objetos Python e texto JSON.

| Função | Descrição |
|--------|-----------|
| `json.dump(obj, f)` | Salva objeto Python em arquivo JSON |
| `json.load(f)` | Carrega JSON de arquivo para objeto Python |
| `json.dumps(obj)` | Serializa para string JSON |
| `json.loads(s)` | Converte string JSON para objeto Python |

In [ ]:
import json

# Salvando dados em JSON
turma = {
    "turma": "Python Iniciante",
    "semestre": "2025-1",
    "alunos": [
        {"id": 1, "nome": "Ana",     "nota": 8.5, "aprovado": True},
        {"id": 2, "nome": "Bruno",   "nota": 6.0, "aprovado": False},
        {"id": 3, "nome": "Carlos",  "nota": 9.2, "aprovado": True},
        {"id": 4, "nome": "Diana",   "nota": 7.0, "aprovado": True},
        {"id": 5, "nome": "Eduardo", "nota": 4.5, "aprovado": False},
    ]
}

with open('turma.json', 'w', encoding='utf-8') as f:
    json.dump(turma, f, ensure_ascii=False, indent=2)

print('turma.json criado!')

In [ ]:
# Lendo o JSON de volta
with open('turma.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

print(f"Turma:    {dados['turma']}")
print(f"Semestre: {dados['semestre']}")
print(f"Alunos:   {len(dados['alunos'])}")
print()

for aluno in dados['alunos']:
    status = 'Aprovado' if aluno['aprovado'] else 'Reprovado'
    print(f"  {aluno['id']:>2}. {aluno['nome']:<10} {aluno['nota']:.1f}  {status}")

In [ ]:
# Adicionando um aluno ao JSON e salvando
with open('turma.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

novo_aluno = {"id": 6, "nome": "Fernanda", "nota": 8.8, "aprovado": True}
dados['alunos'].append(novo_aluno)

with open('turma.json', 'w', encoding='utf-8') as f:
    json.dump(dados, f, ensure_ascii=False, indent=2)

print(f'Total de alunos agora: {len(dados["alunos"])}')

In [ ]:
# json.dumps e json.loads — conversão para/de string
produto = {'codigo': 'P001', 'nome': 'Caderno', 'preco': 12.50}

texto_json = json.dumps(produto, ensure_ascii=False)
print(f'String JSON: {texto_json}')
print(f'Tipo:        {type(texto_json)}')

# Convertendo de volta
objeto = json.loads(texto_json)
print(f'\nObjeto Python: {objeto}')
print(f'Preço:         R$ {objeto["preco"]:.2f}')

---
## 5. Manipulação de Tabelas

Com os dados carregados como lista de dicionários, podemos aplicar operações de tabela:
- **Filtrar** — selecionar registros por condição
- **Ordenar** — classificar por um ou mais campos
- **Projetar** — selecionar apenas certas colunas
- **Agregar** — calcular totais, médias, contagens por grupo

In [ ]:
# Criando uma tabela de vendas para os exemplos
import csv

vendas_dados = [
    ['id', 'vendedor',  'produto',    'categoria', 'quantidade', 'preco_unit', 'cidade'],
    [1,  'Ana',       'Notebook',   'Eletrônico',  2,  3500.00, 'Rio de Janeiro'],
    [2,  'Bruno',     'Mouse',      'Eletrônico',  5,    80.00, 'São Paulo'],
    [3,  'Ana',       'Cadeira',    'Móvel',       1,   800.00, 'Rio de Janeiro'],
    [4,  'Carlos',    'Monitor',    'Eletrônico',  3,  1200.00, 'Belo Horizonte'],
    [5,  'Bruno',     'Mesa',       'Móvel',       2,   450.00, 'São Paulo'],
    [6,  'Diana',     'Teclado',    'Eletrônico',  8,   150.00, 'Rio de Janeiro'],
    [7,  'Carlos',    'Notebook',   'Eletrônico',  1,  3500.00, 'Belo Horizonte'],
    [8,  'Ana',       'Mouse',      'Eletrônico', 10,    80.00, 'Rio de Janeiro'],
    [9,  'Diana',     'Cadeira',    'Móvel',       3,   800.00, 'Curitiba'],
    [10, 'Bruno',     'Monitor',    'Eletrônico',  2,  1200.00, 'São Paulo'],
    [11, 'Carlos',    'Mesa',       'Móvel',       4,   450.00, 'Belo Horizonte'],
    [12, 'Ana',       'Teclado',    'Eletrônico',  6,   150.00, 'Rio de Janeiro'],
]

with open('vendas.csv', 'w', newline='', encoding='utf-8') as f:
    csv.writer(f).writerows(vendas_dados)

print('vendas.csv criado!')

In [ ]:
# Carregando e convertendo tipos
def carregar_vendas(caminho):
    vendas = []
    with open(caminho, 'r', encoding='utf-8') as f:
        for row in csv.DictReader(f):
            vendas.append({
                'id':          int(row['id']),
                'vendedor':    row['vendedor'],
                'produto':     row['produto'],
                'categoria':   row['categoria'],
                'quantidade':  int(row['quantidade']),
                'preco_unit':  float(row['preco_unit']),
                'cidade':      row['cidade'],
                'total':       int(row['quantidade']) * float(row['preco_unit'])
            })
    return vendas

vendas = carregar_vendas('vendas.csv')
print(f'Total de vendas: {len(vendas)}')
print(f'Exemplo: {vendas[0]}')

### 5.1 Filtrar — WHERE

In [ ]:
# Filtrar vendas da categoria Eletrônico
eletronicos = [v for v in vendas if v['categoria'] == 'Eletrônico']

print(f'Vendas de Eletrônicos ({len(eletronicos)}):')          
for v in eletronicos:
    print(f"  {v['vendedor']:<8} {v['produto']:<10} R$ {v['total']:>8,.2f}")

In [ ]:
# Filtrar vendas com total acima de R$ 2.000
grandes = [v for v in vendas if v['total'] > 2000]

print('Vendas acima de R$ 2.000:')
for v in grandes:
    print(f"  ID {v['id']:>2} | {v['vendedor']:<8} | {v['produto']:<10} | R$ {v['total']:>8,.2f}")

In [ ]:
# Filtro com múltiplas condições (AND / OR)
filtro_and = [v for v in vendas if v['vendedor'] == 'Ana' and v['categoria'] == 'Eletrônico']
filtro_or  = [v for v in vendas if v['cidade'] == 'São Paulo' or v['cidade'] == 'Curitiba']

print(f'Vendas da Ana em Eletrônico: {len(filtro_and)}')
for v in filtro_and:
    print(f"  {v['produto']:<10} R$ {v['total']:>8,.2f}")

print(f'\nVendas em SP ou Curitiba: {len(filtro_or)}')
for v in filtro_or:
    print(f"  {v['cidade']:<20} {v['produto']:<10} R$ {v['total']:>8,.2f}")

### 5.2 Ordenar — ORDER BY

In [ ]:
# Ordenar por total (maior para menor)
por_total = sorted(vendas, key=lambda v: v['total'], reverse=True)

print('Ranking de vendas (maior → menor):')
for i, v in enumerate(por_total, 1):
    print(f"  {i:>2}º {v['vendedor']:<8} {v['produto']:<10} R$ {v['total']:>8,.2f}")

In [ ]:
# Ordenar por múltiplos campos: vendedor (A→Z) e total (maior→menor)
from operator import itemgetter

# Usando sorted com múltiplas chaves
ordenado = sorted(vendas, key=lambda v: (v['vendedor'], -v['total']))

print(f"{'Vendedor':<10} {'Produto':<12} {'Total':>10}")
print('-' * 35)
for v in ordenado:
    print(f"{v['vendedor']:<10} {v['produto']:<12} R$ {v['total']:>7,.2f}")

### 5.3 Projetar — SELECT (escolher colunas)

In [ ]:
# Selecionar apenas as colunas necessárias
def projetar(tabela, campos):
    """Retorna apenas os campos indicados de cada registro."""
    return [{campo: registro[campo] for campo in campos} for registro in tabela]

resumo = projetar(vendas, ['vendedor', 'produto', 'total'])

for r in resumo[:5]:
    print(r)

### 5.4 Agregar — GROUP BY

In [ ]:
# Total de vendas por vendedor
def agrupar_soma(tabela, campo_grupo, campo_valor):
    """Agrupa registros e soma o campo_valor por campo_grupo."""
    grupos = {}
    for registro in tabela:
        chave = registro[campo_grupo]
        grupos[chave] = grupos.get(chave, 0) + registro[campo_valor]
    return grupos

total_por_vendedor = agrupar_soma(vendas, 'vendedor', 'total')

print('Total de vendas por vendedor:')
for vendedor, total in sorted(total_por_vendedor.items(), key=lambda x: -x[1]):
    print(f"  {vendedor:<10} R$ {total:>9,.2f}")

In [ ]:
# Contagem por categoria
def agrupar_contagem(tabela, campo_grupo):
    contagem = {}
    for registro in tabela:
        chave = registro[campo_grupo]
        contagem[chave] = contagem.get(chave, 0) + 1
    return contagem

por_categoria = agrupar_contagem(vendas, 'categoria')
por_cidade    = agrupar_soma(vendas, 'cidade', 'total')

print('Vendas por categoria:')
for cat, qtd in por_categoria.items():
    print(f"  {cat}: {qtd} venda(s)")

print('\nFaturamento por cidade:')
for cidade, total in sorted(por_cidade.items(), key=lambda x: -x[1]):
    print(f"  {cidade:<20} R$ {total:>9,.2f}")

In [ ]:
# Estatísticas completas por grupo
def estatisticas_por_grupo(tabela, campo_grupo, campo_valor):
    """Retorna soma, média, máximo, mínimo e contagem por grupo."""
    grupos = {}
    for registro in tabela:
        chave = registro[campo_grupo]
        val   = registro[campo_valor]
        if chave not in grupos:
            grupos[chave] = []
        grupos[chave].append(val)

    resultado = {}
    for chave, valores in grupos.items():
        resultado[chave] = {
            'count': len(valores),
            'soma':  sum(valores),
            'media': sum(valores) / len(valores),
            'maior': max(valores),
            'menor': min(valores),
        }
    return resultado

stats = estatisticas_por_grupo(vendas, 'vendedor', 'total')

print(f"{'Vendedor':<10} {'Qtd':>5} {'Soma':>12} {'Média':>10} {'Maior':>10}")
print('-' * 52)
for vendedor, s in sorted(stats.items()):
    print(f"{vendedor:<10} {s['count']:>5} R$ {s['soma']:>9,.2f} R$ {s['media']:>7,.2f} R$ {s['maior']:>7,.2f}")

### 5.5 Salvar tabela processada em CSV

In [ ]:
# Salvando o relatório de vendedor em CSV
def salvar_csv(caminho, dados, campos):
    """Salva lista de dicionários em arquivo CSV."""
    with open(caminho, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=campos, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dados)
    print(f'{len(dados)} registros salvos em {caminho}')

# Criar lista de dicionários do relatório por vendedor
relatorio = [
    {'vendedor': v, 'qtd_vendas': s['count'], 'total': round(s['soma'], 2), 'media': round(s['media'], 2)}
    for v, s in sorted(stats.items(), key=lambda x: -x[1]['soma'])
]

salvar_csv('relatorio_vendedores.csv', relatorio, ['vendedor', 'qtd_vendas', 'total', 'media'])

---
## 6. Projeto Prático — Sistema de Cadastro de Clientes

Vamos unir tudo que aprendemos em um sistema completo que:
- Usa JSON como banco de dados em arquivo
- Indexa por ID para busca rápida
- Permite **criar**, **ler**, **atualizar** e **excluir** registros (CRUD)
- Exporta relatórios em CSV

In [ ]:
import json

ARQUIVO_CLIENTES = 'clientes.json'

def _carregar():
    """Carrega os dados do arquivo JSON."""
    try:
        with open(ARQUIVO_CLIENTES, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        return {'proximo_id': 1, 'clientes': {}}

def _salvar(dados):
    """Salva os dados no arquivo JSON."""
    with open(ARQUIVO_CLIENTES, 'w', encoding='utf-8') as f:
        json.dump(dados, f, ensure_ascii=False, indent=2)

# ── CREATE ───────────────────────────────────────────────────────────────────
def criar_cliente(nome, email, cidade):
    """Adiciona um novo cliente e retorna o ID gerado."""
    dados = _carregar()
    novo_id = str(dados['proximo_id'])
    dados['clientes'][novo_id] = {
        'id': novo_id, 'nome': nome, 'email': email, 'cidade': cidade
    }
    dados['proximo_id'] += 1
    _salvar(dados)
    return novo_id

# ── READ ─────────────────────────────────────────────────────────────────────
def buscar_cliente(id_cliente):
    """Busca cliente por ID (acesso O(1) pelo índice)."""
    dados = _carregar()
    return dados['clientes'].get(str(id_cliente))

def listar_clientes():
    """Retorna todos os clientes como lista."""
    dados = _carregar()
    return list(dados['clientes'].values())

# ── UPDATE ───────────────────────────────────────────────────────────────────
def atualizar_cliente(id_cliente, **campos):
    """Atualiza campos do cliente. Retorna True se encontrado."""
    dados = _carregar()
    chave = str(id_cliente)
    if chave not in dados['clientes']:
        return False
    dados['clientes'][chave].update(campos)
    _salvar(dados)
    return True

# ── DELETE ───────────────────────────────────────────────────────────────────
def excluir_cliente(id_cliente):
    """Remove um cliente. Retorna True se encontrado."""
    dados = _carregar()
    chave = str(id_cliente)
    if chave not in dados['clientes']:
        return False
    del dados['clientes'][chave]
    _salvar(dados)
    return True

print('Funções CRUD carregadas!')

In [ ]:
# Criando clientes
import os
if os.path.exists(ARQUIVO_CLIENTES):
    os.remove(ARQUIVO_CLIENTES)   # limpa dados de execuções anteriores

id1 = criar_cliente('Ana Silva',    'ana@email.com',    'Rio de Janeiro')
id2 = criar_cliente('Bruno Costa',  'bruno@email.com',  'São Paulo')
id3 = criar_cliente('Carla Souza',  'carla@email.com',  'Belo Horizonte')
id4 = criar_cliente('Diego Lima',   'diego@email.com',  'Curitiba')
id5 = criar_cliente('Eva Rocha',    'eva@email.com',    'Rio de Janeiro')

print(f'IDs criados: {id1}, {id2}, {id3}, {id4}, {id5}')

In [ ]:
# Listando todos
print(f"{'ID':<5} {'Nome':<15} {'E-mail':<25} {'Cidade'}")
print('-' * 60)
for c in listar_clientes():
    print(f"{c['id']:<5} {c['nome']:<15} {c['email']:<25} {c['cidade']}")

In [ ]:
# Buscando por ID (indexado — O(1))
cliente = buscar_cliente(3)
print(f'Cliente ID 3: {cliente}')

nao_existe = buscar_cliente(99)
print(f'Cliente ID 99: {nao_existe}')

In [ ]:
# Atualizando
sucesso = atualizar_cliente(2, email='bruno.costa@nova.com', cidade='Campinas')
print(f'Atualização ID 2: {sucesso}')
print(f'Novo estado: {buscar_cliente(2)}')

In [ ]:
# Excluindo
print(f'Antes: {len(listar_clientes())} clientes')
excluir_cliente(4)
print(f'Após excluir ID 4: {len(listar_clientes())} clientes')

In [ ]:
# Exportando para CSV
clientes = listar_clientes()
salvar_csv('clientes_export.csv', clientes, ['id', 'nome', 'email', 'cidade'])

---
## 7. Exercícios Práticos

Resolva os exercícios abaixo utilizando os arquivos gerados ao longo da aula.

### Exercício 1 — Contar linhas de um arquivo

Crie uma função `contar_linhas(caminho)` que abra um arquivo de texto e retorne o número de linhas que **não estão vazias**.

In [ ]:
def contar_linhas(caminho):
    with open(caminho, 'r', encoding='utf-8') as f:
        return sum(1 for linha in f if linha.strip())

print(f'notas.txt:  {contar_linhas("notas.txt")} linhas')
print(f'alunos.csv: {contar_linhas("alunos.csv")} linhas (incluindo cabeçalho)')

### Exercício 2 — Busca em CSV por campo

Crie uma função `pesquisar_csv(caminho, campo, valor)` que leia o CSV e retorne todos os registros onde `campo` contém `valor` (busca parcial, sem diferenciar maiúsculas/minúsculas).

In [ ]:
def pesquisar_csv(caminho, campo, valor):
    with open(caminho, 'r', encoding='utf-8') as f:
        return [
            row for row in csv.DictReader(f)
            if valor.lower() in row[campo].lower()
        ]

# Testar
resultado = pesquisar_csv('alunos.csv', 'cidade', 'são paulo')
print(f'Alunos em São Paulo: {len(resultado)}')
for r in resultado:
    print(f"  {r}")

### Exercício 3 — Copiar e filtrar CSV

Leia o `vendas.csv`, filtre apenas as vendas com total acima de R$ 1.000 e salve o resultado em `grandes_vendas.csv`.

In [ ]:
vendas = carregar_vendas('vendas.csv')
grandes = [v for v in vendas if v['total'] > 1000]

salvar_csv(
    'grandes_vendas.csv',
    grandes,
    ['id', 'vendedor', 'produto', 'categoria', 'quantidade', 'preco_unit', 'cidade', 'total']
)

print(f'\nVendas filtradas:')
for v in grandes:
    print(f"  {v['vendedor']:<8} {v['produto']:<10} R$ {v['total']:>8,.2f}")

### Exercício 4 — Relatório de notas a partir do arquivo

Leia o arquivo `notas.txt` (formato `Nome: nota`), calcule a média e exiba quem foi aprovado (nota >= 7) e reprovado.

In [ ]:
aprovados  = []
reprovados = []

with open('notas.txt', 'r', encoding='utf-8') as f:
    for linha in f:
        nome, nota = linha.strip().split(': ')
        nota = float(nota)
        if nota >= 7:
            aprovados.append((nome, nota))
        else:
            reprovados.append((nome, nota))

todos = aprovados + reprovados
media_turma = sum(n for _, n in todos) / len(todos)

print(f'Aprovados ({len(aprovados)}):')
for nome, nota in sorted(aprovados, key=lambda x: -x[1]):
    print(f'  {nome:<10} {nota:.1f}')

print(f'\nReprovados ({len(reprovados)}):')
for nome, nota in reprovados:
    print(f'  {nome:<10} {nota:.1f}')

print(f'\nMédia da turma: {media_turma:.2f}')

### Exercício 5 — Tabela de frequência

A partir do `alunos.csv`, conte quantos alunos há em cada cidade e exiba o resultado em ordem decrescente de frequência.

In [ ]:
alunos = carregar_csv('alunos.csv')

frequencia = {}
for aluno in alunos:
    cidade = aluno['cidade']
    frequencia[cidade] = frequencia.get(cidade, 0) + 1

print(f"{'Cidade':<25} {'Alunos':>8}")
print('-' * 35)
for cidade, qtd in sorted(frequencia.items(), key=lambda x: -x[1]):
    barra = '█' * qtd
    print(f'{cidade:<25} {qtd:>5}  {barra}')

### Exercício 6 — Mesclar dois arquivos CSV

Você tem dois arquivos de alunos separados. Mescle-os em um único arquivo, removendo duplicatas pelo campo `nome`.

In [ ]:
# Criar dois arquivos de teste
with open('turma_a.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows([
        ['nome', 'nota'],
        ['Ana', 8.5], ['Bruno', 6.0], ['Carlos', 9.2]
    ])

with open('turma_b.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows([
        ['nome', 'nota'],
        ['Diana', 7.0], ['Bruno', 7.5], ['Eduardo', 5.5]  # Bruno duplicado
    ])

# Mesclar sem duplicatas
def mesclar_csv(arquivo1, arquivo2, saida, chave_unica):
    registros = {}
    for arq in [arquivo1, arquivo2]:
        with open(arq, 'r', encoding='utf-8') as f:
            for row in csv.DictReader(f):
                chave = row[chave_unica]
                if chave not in registros:   # mantém o primeiro encontrado
                    registros[chave] = row

    todos = list(registros.values())
    if todos:
        salvar_csv(saida, todos, list(todos[0].keys()))
    return todos

resultado = mesclar_csv('turma_a.csv', 'turma_b.csv', 'turma_unificada.csv', 'nome')
print('\nRegistros unificados:')
for r in resultado:
    print(f"  {r}")

### Exercício 7 — Atualizar campo em CSV

Crie uma função que leia o `alunos.csv`, atualize a nota de um aluno específico pelo nome e salve o arquivo atualizado.

In [ ]:
def atualizar_nota_csv(caminho, nome_aluno, nova_nota):
    """Atualiza a nota de um aluno no CSV."""
    alunos = carregar_csv(caminho)
    encontrado = False

    for aluno in alunos:
        if aluno['nome'].lower() == nome_aluno.lower():
            aluno['nota'] = str(nova_nota)
            encontrado = True
            break

    if encontrado:
        salvar_csv(caminho, alunos, list(alunos[0].keys()))
        print(f"Nota de '{nome_aluno}' atualizada para {nova_nota}")
    else:
        print(f"Aluno '{nome_aluno}' não encontrado.")

atualizar_nota_csv('alunos.csv', 'Ana', 9.5)
atualizar_nota_csv('alunos.csv', 'Maria', 8.0)   # não existe

# Verificar
ana = next((a for a in carregar_csv('alunos.csv') if a['nome'] == 'Ana'), None)
print(f'\nAna após atualização: {ana}')

### Exercício 8 — Converter CSV para JSON

Leia o `vendas.csv` e salve os dados como `vendas.json`, incluindo o campo `total` calculado (`quantidade × preco_unit`).

In [ ]:
def csv_para_json(caminho_csv, caminho_json):
    vendas = carregar_vendas(caminho_csv)
    with open(caminho_json, 'w', encoding='utf-8') as f:
        json.dump(vendas, f, ensure_ascii=False, indent=2)
    print(f'{len(vendas)} registros convertidos para {caminho_json}')

csv_para_json('vendas.csv', 'vendas.json')

# Verificar
with open('vendas.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

print(f'\nPrimeiro registro do JSON:')
print(json.dumps(dados[0], ensure_ascii=False, indent=2))

### Exercício 9 — Top 3 produtos mais vendidos

A partir do `vendas.csv`, calcule a quantidade total vendida por produto e exiba o top 3.

In [ ]:
vendas = carregar_vendas('vendas.csv')

# Agrupar quantidade por produto
qtd_por_produto = {}
for v in vendas:
    produto = v['produto']
    qtd_por_produto[produto] = qtd_por_produto.get(produto, 0) + v['quantidade']

# Ordenar e pegar top 3
top3 = sorted(qtd_por_produto.items(), key=lambda x: -x[1])[:3]

print('Top 3 produtos mais vendidos (em quantidade):')
for pos, (produto, qtd) in enumerate(top3, 1):
    print(f'  {pos}º {produto:<12} {qtd} unidades')

### Exercício 10 — Log de operações

Crie uma função `registrar_log(mensagem)` que adicione ao arquivo `operacoes.log` uma linha com data/hora e a mensagem. Depois, teste registrando 3 operações e exiba o conteúdo do log.

In [ ]:
from datetime import datetime

def registrar_log(mensagem, arquivo='operacoes.log'):
    """Adiciona uma entrada de log com timestamp."""
    agora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(arquivo, 'a', encoding='utf-8') as f:
        f.write(f'[{agora}] {mensagem}\n')

registrar_log('Sistema iniciado')
registrar_log('Cliente ID 3 consultado')
registrar_log('Relatório de vendas gerado')

print('Conteúdo do log:')
with open('operacoes.log', 'r', encoding='utf-8') as f:
    print(f.read())

---
## 8. Resumo

### Operações com arquivos

```python
# Texto
with open('arq.txt', 'r', encoding='utf-8') as f:   # ler
    conteudo = f.read()           # tudo de uma vez
    linhas   = f.readlines()      # lista de linhas
    for linha in f: ...           # linha a linha

with open('arq.txt', 'w', encoding='utf-8') as f:   # escrever
    f.write('texto\n')

with open('arq.txt', 'a', encoding='utf-8') as f:   # acrescentar
    f.write('mais texto\n')
```

### CSV

```python
import csv

# Ler
with open('tabela.csv', 'r', encoding='utf-8') as f:
    for row in csv.DictReader(f):   # row é um dicionário
        print(row['nome'])

# Escrever
with open('tabela.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['nome', 'nota'])
    writer.writeheader()
    writer.writerows(lista_de_dicts)
```

### JSON

```python
import json

# Salvar
with open('dados.json', 'w', encoding='utf-8') as f:
    json.dump(objeto_python, f, ensure_ascii=False, indent=2)

# Carregar
with open('dados.json', 'r', encoding='utf-8') as f:
    objeto_python = json.load(f)
```

### Manipulação de tabelas (lista de dicionários)

| Operação | Código Python |
|----------|---------------|
| Filtrar | `[r for r in tabela if r['campo'] == valor]` |
| Ordenar | `sorted(tabela, key=lambda r: r['campo'])` |
| Projetar | `[{c: r[c] for c in campos} for r in tabela]` |
| Agregar | Usar dicionário acumulador com `.get(chave, 0)` |
| Indexar | `{r['id']: r for r in tabela}` → acesso O(1) |

### Boas práticas
- Sempre use `with open(...)` — garante o fechamento do arquivo
- Especifique `encoding='utf-8'` para evitar problemas com acentuação
- Use `newline=''` no `open()` ao escrever CSV no Windows
- Converta tipos (`int`, `float`) ao carregar CSV — tudo chega como `str`
- Prefira `DictReader`/`DictWriter` ao trabalhar com CSV — código mais legível
- Use JSON para dados hierárquicos; CSV para dados tabulares simples
- Crie índices em dicionário quando precisar de buscas frequentes por chave